In [114]:
import torch
from sampling import cal_weight

In [115]:
B,N = 4,20
xyz=torch.randn(4, 20,3)
print("xyz:",xyz.shape)

xyz: torch.Size([4, 20, 3])


In [112]:
def filter_random_point_sample(xyz, npoint, k=20,replace=False):
    """
    Input:
        xyz: pointcloud data, [B, N, C]
        npoint: number of samples
        k: k of knn
    Return:
        centroids: sampled pointcloud index, [B, npoint]
    """
    # print("k::::",k)
    device = xyz.device
    B, N, C = xyz.shape	
    centroids = torch.zeros(B, npoint, dtype=torch.long)#.to(device)
    # 计算采样权重
    num=int(N*0.95)
    print("num:",num)
    # print("xyz:",xyz)
    weights, idx = cal_weight(xyz, k) #需要是(B,N)
    print("weights:",weights)
    # print("weights:",weights)
    # 使用 torch.multinomial 来进行带权重的随机采样
    # print("replace:",replace)
    indices = torch.arange(N, device=weights.device).unsqueeze(0).repeat(B, 1)  
    # 将权重和索引堆叠起来，以便后续排序  
    weighted_indices = torch.stack([weights, indices], dim=-1)  
    # 根据权重对堆叠后的张量进行排序，获取排序后的索引  
    _, sorted_indices = torch.sort(weighted_indices[:, :, 0], dim=1,descending=True)  
    # 选择前num个 
    smallest_indices = sorted_indices[:,:num]  
    # 创建一个与原weights形状相同的mask张量，初始化全为False  
    mask = torch.zeros(B, N, dtype=torch.bool, device=weights.device)  
    # 使用scatter_将smallest_indices对应的位置在mask中设置为True  
    mask.scatter_(1, smallest_indices, 1) 
    # print("mask:",mask)
    modified_weights = (mask).float()
    print("modified_weights:",modified_weights)
    centroids = torch.multinomial(modified_weights, npoint,replacement=replace).squeeze()
    # print("centroids:",centroids.shape)
    return centroids,idx

In [113]:
cen,_ = filter_random_point_sample(xyz,10,3)
print("cen:",cen)

num: 19
weights: tensor([[0.0625, 0.0312, 0.0312, 0.0625, 0.0312, 0.0625, 0.0625, 0.0938, 0.0312,
         0.0312, 0.0312, 0.0312, 0.0312, 0.0312, 0.0625, 0.0625, 0.0625, 0.0312,
         0.0938, 0.0625],
        [0.0556, 0.0556, 0.0556, 0.0278, 0.0556, 0.0278, 0.0833, 0.0556, 0.0556,
         0.0278, 0.0278, 0.0278, 0.0278, 0.0833, 0.0556, 0.0556, 0.0556, 0.0556,
         0.0833, 0.0278],
        [0.0882, 0.0294, 0.0588, 0.0294, 0.0588, 0.0294, 0.0294, 0.0588, 0.0882,
         0.0294, 0.0588, 0.0588, 0.0588, 0.0588, 0.0294, 0.0294, 0.0294, 0.0294,
         0.0882, 0.0588],
        [0.0857, 0.0857, 0.0571, 0.0571, 0.0286, 0.0286, 0.0286, 0.0857, 0.0286,
         0.0286, 0.0286, 0.0571, 0.0286, 0.0857, 0.0571, 0.0857, 0.0571, 0.0286,
         0.0286, 0.0286]])
modified_weights: tensor([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.],
        [1., 0

In [116]:
num1 = int(N * 0.8)
num2 = int(N*0.8)
weights,idx_k = cal_weight(xyz, k=5)

In [117]:
print("weights:",weights)

weights: tensor([[0.0385, 0.0769, 0.0577, 0.0192, 0.0769, 0.0962, 0.0385, 0.0769, 0.0385,
         0.0769, 0.0577, 0.0192, 0.0192, 0.0385, 0.0192, 0.0769, 0.0192, 0.0385,
         0.0192, 0.0962],
        [0.0862, 0.0172, 0.0862, 0.0690, 0.0690, 0.0862, 0.0345, 0.0172, 0.0345,
         0.0345, 0.0862, 0.0862, 0.0345, 0.0517, 0.0345, 0.0345, 0.0517, 0.0345,
         0.0345, 0.0172],
        [0.0455, 0.0227, 0.0682, 0.0909, 0.0227, 0.0909, 0.0909, 0.0682, 0.0455,
         0.0227, 0.0455, 0.0455, 0.0227, 0.0227, 0.0227, 0.0682, 0.0455, 0.0227,
         0.0682, 0.0682],
        [0.0377, 0.0755, 0.0377, 0.0377, 0.0755, 0.0755, 0.0377, 0.0755, 0.0943,
         0.0377, 0.0755, 0.0189, 0.0189, 0.0377, 0.0755, 0.0377, 0.0566, 0.0377,
         0.0377, 0.0189]])


In [118]:
# 创建一个索引张量  
indices = torch.arange(N, device=weights.device).unsqueeze(0).repeat(B, 1)  
# 将权重和索引堆叠起来，以便后续排序  
weighted_indices = torch.stack([weights, indices], dim=-1)  
# 根据权重对堆叠后的张量进行排序，获取排序后的索引  
_, sorted_indices = torch.sort(weighted_indices[:, :, 0], dim=1,descending=True)  
# 选择前num个 
smallest_indices = sorted_indices[:, num1:] 
print("smallest_indices:",smallest_indices) 
# 创建一个与原weights形状相同的mask张量，初始化全为False  
mask = torch.zeros(B, N, dtype=torch.bool, device=weights.device)  
# 使用scatter_将smallest_indices对应的位置在mask中设置为True  
mask.scatter_(1, smallest_indices, 1) 
# print("mask:",mask)
biggest_indices =  sorted_indices[:, :N-num2]
print("biggest_indices:",biggest_indices)
mask.scatter_(1, biggest_indices, 1) 

# 将mask中标记为True的位置在weights中设置为-1  
print("before weights:",weights)
weights[mask] = -1  
print("after weights:",weights)
print("N-num:",(N-num1)*B)
print(torch.sum(weights==-1))

smallest_indices: tensor([[12, 14, 16, 18],
        [ 6, 19,  7,  1],
        [ 9, 17,  4,  1],
        [18, 11, 12, 19]])
biggest_indices: tensor([[19,  5,  1, 15,  9],
        [ 0,  2,  5, 10, 11],
        [ 3,  5,  6, 19,  2],
        [ 8,  1, 14, 10,  7]])
before weights: tensor([[0.0385, 0.0769, 0.0577, 0.0192, 0.0769, 0.0962, 0.0385, 0.0769, 0.0385,
         0.0769, 0.0577, 0.0192, 0.0192, 0.0385, 0.0192, 0.0769, 0.0192, 0.0385,
         0.0192, 0.0962],
        [0.0862, 0.0172, 0.0862, 0.0690, 0.0690, 0.0862, 0.0345, 0.0172, 0.0345,
         0.0345, 0.0862, 0.0862, 0.0345, 0.0517, 0.0345, 0.0345, 0.0517, 0.0345,
         0.0345, 0.0172],
        [0.0455, 0.0227, 0.0682, 0.0909, 0.0227, 0.0909, 0.0909, 0.0682, 0.0455,
         0.0227, 0.0455, 0.0455, 0.0227, 0.0227, 0.0227, 0.0682, 0.0455, 0.0227,
         0.0682, 0.0682],
        [0.0377, 0.0755, 0.0377, 0.0377, 0.0755, 0.0755, 0.0377, 0.0755, 0.0943,
         0.0377, 0.0755, 0.0189, 0.0189, 0.0377, 0.0755, 0.0377, 0.0566, 0.037

In [47]:
indices = torch.arange(N, device=weights.device).unsqueeze(0).repeat(B, 1)


In [48]:
print("indices:",indices.shape)

indices: torch.Size([4, 20])


In [49]:
sorted_weights_indices = torch.stack([weights, indices], dim=-1)  

In [50]:
print("sorted_weights_indices:",sorted_weights_indices.shape)

sorted_weights_indices: torch.Size([4, 20, 2])


In [51]:
sorted_weights_indices_sorted, _ = torch.sort(sorted_weights_indices, dim=1, descending=False)
print("sorted_weights_indices_sorted:",sorted_weights_indices_sorted.shape)   

sorted_weights_indices_sorted: torch.Size([4, 20, 2])


In [52]:
sorted_weights, sorted_indices = sorted_weights_indices_sorted[:, :, 0], sorted_weights_indices_sorted[:, :, 1]  
sorted_indices = sorted_indices.to(torch.int64)  
print("sorted_indices:",sorted_indices.shape)

sorted_indices: torch.Size([4, 20])


In [53]:
mask = torch.zeros(B, N, dtype=torch.bool, device=weights.device)  
keep_num = N - num 
print("keep:",keep_num)
mask.scatter_(1, sorted_indices[:, :keep_num].contiguous(), 1)  
print("mask:",mask)
# mask = ~mask  # 反转mask，因为我们之前设置的是要保留的位置，现在要得到要设置为-1的位置  
# print("mask:",mask)
# print("before sort:",weights)
print("before weights:",sorted_weights)
# 使用masked_fill_将mask为True的位置在weights中设置为-1  
sorted_weights.masked_fill_(mask, -1)
print("sorted_weights:",sorted_weights)

keep: 5
mask: tensor([[ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]])
before weights: tensor([[-1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  0.0351,  0.0351,  0.0351,
          0.0351,  0.0351,  0.0526,  0.0526,  0.0702,  0.0702,  0.0702,  0.0702,
          0.0877,  0.0877,  0.0877,  0.0877],
        [-1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  0.0364,  0.0364,  0.0364,
          0.0364,  0.0364,  0.0364,  0.0364,

In [54]:
r = torch.normal(mean=torch.tensor([1.0]), std=torch.tensor(0.3))  # Gaussian distribution with mean=1.0, std=0.1
print(r)
if r <= 1:
    # Downsample the point cloud
    k = torch.clamp(r,0.5, 1.0)  # Clip ratio to be within [0.75/0.5, 1.0]
    ratio = 2**(k-1)
    print("k:",k)
    print("ratio:",ratio)
    print("2**r:",2**(r-1))
else:
    k = torch.clamp(r, 1.0,2.0)  # Clip ratio to be within [0.75/0.5, 1.0]
    ratio = 2**(k-1)
    print("k:",k)
    print("ratio:",ratio)
    print("2**r:",2**(r-1))

rat = 2**((torch.rand(1) - 0.5)*2)
print("rat:",rat)

tensor([1.2429])
k: tensor([1.2429])
ratio: tensor([1.1834])
2**r: tensor([1.1834])
rat: tensor([0.5946])


In [55]:
import torch  
  
# 生成与 2**((torch.rand(1) - 0.5) * 2) 分布相同的随机数  
uniform_sample = torch.rand(1)  # 生成 [0, 1) 范围内的随机数  
rat = 2 ** ((uniform_sample - 0.5) * 2)  # 将随机数映射到 [0.5, 2] 范围内  
  
print("rat:", rat)  
  
# 根据 rat 的值进行后续处理  
if rat <= 1:  
    # 下采样逻辑  
    k = torch.clamp(rat, 0.5, 1.0)  # 限制 k 的范围在 [0.5, 1.0]  
    ratio = 2 ** (k - 1)  # 根据 k 计算 ratio  
    print("k:", k)  
    print("ratio:", ratio)  
else:  
    k = torch.clamp(rat, 1.0, 2.0)  # 如果 rat > 1，则限制 k 的范围在 [1.0, 2.0]  
    ratio = 2 ** (k - 1)  # 根据 k 计算 ratio  
    print("k:", k)  
    print("ratio:", ratio)

rat: tensor([0.5821])
k: tensor([0.5821])
ratio: tensor([0.7485])


In [56]:
import plotly.io as pio
#pio.renderers.default='colab'
import plotly.graph_objects as go
import numpy as np
#生成一些示例点云数据
point_cloud = filtered_point_cloud[0]
# 使用Plotly进行3D可视化
fig =go.Figure(data=[go.Scatter3d(
x=point cloud[:,0],
y=point cloud[:,1],
z=point cloud[:,2],
mode='markers',
marker=dict(
size=2,
color=point_cloud[:,2],# 将颜色设置为Z轴的值
colorscale='Viridis'))])

SyntaxError: invalid syntax (3437293911.py, line 9)